In [172]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

def simulate_steps(duration=10.0, resolution=1000.0, dwell_time=1.0,
                   step_size=8.0, SNR=0.5, movement='diffusive',
                   constant_dwell=False):
    """
    Simulate data of steps with noise.

    Parameters
    ----------
    duration : float
        Duration of whole trace in seconds.
    resolution : float
        Resolution of the trace in data points / second.
    dwell_time : float
        Time between steps in seconds.
    step_size : float
        The size of a step.
    SNR : float
        Signal to noise ratio, i.e. the size of a step divided by the standard
        deviation of the noise.
    movement : str
        'monoton' or 'diffusive' movement.
    constant_dwell : bool
        Constant or exponential distributed dwell times.

    Returns
    -------
    StepsSimulated : namedtuple
    """
    # Number of data points between steps
    dwell_points = np.round(dwell_time * resolution).astype(int)

    # Number of data points in whole trajectory
    length = np.round(duration * resolution).astype(int)

    # Random steps with constant or exponential distributed dwell times
    if movement == 'monoton' and constant_dwell:
        data = step_size * np.floor(np.arange(0, length) / dwell_points)
        dwells = np.full(length + 1, dwell_points, dtype=np.int64)
        indices = np.arange(dwell_points, length, dwell_points)
    else:
        data = np.empty(length)
        height = 0
        i = 0
        step = 0
        points = dwell_points
        dwells = np.empty(0, dtype=np.int64)
        indices = np.empty(0, dtype=np.int64)
        while i < length:
            if not constant_dwell:
                random = np.random.exponential(scale=dwell_points)
                points = np.ceil(random).astype(int)
            points = min(length - i, points)
            if movement == 'monoton':
                y = np.full(points, step * step_size)
                step += 1
            else:  # 'diffusive':
                sign = 1 #np.random.choice([-1, 1])
                height = height + sign * step_size
                y = np.full(points, height)
            data[i:i + points] = y
            i += points
            dwells = np.r_[dwells, points]
            if i < length:
                indices = np.r_[indices, i]

    # Standard deviation of noise corresponding to SNR set by user
    noise_STD = step_size / SNR
    noise = np.random.normal(scale=noise_STD, size=len(data))

    return (data, resolution, noise, dwells, indices,
                          len(indices))
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


def previous_index(index:int, window:int, length:int, step:int=1):
    # https://stackoverflow.com/questions/56135864/slice-a-list-based-on-an-index-and-items-behind-it-in-python
    return [(index-i * step)%length for i in range(window)]

def NormalizeData(data:np.ndarray):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [219]:
# Set parameters to simulate the steps
duration = 1.0  # s
resolution = 20000.0  # Hz
dwell_time = 0.050  # s
SNR = 0.5

# data, resolution, noise, dwells, indices, indices_len = simulate_steps(duration=duration, resolution=resolution,
#                                     dwell_time=dwell_time, SNR=SNR, constant_dwell=True)
# data_noise = data + noise
data = np.loadtxt("test_data.txt")
data_noise = np.loadtxt("test_data_noise.txt")
# np.savetxt("test_data.txt", data)
# np.savetxt("test_data_noise.txt", data_noise)

# mv_avg = moving_average(data_noise, 100)
# print(data)
# print(data_noise)
plt.plot(data_noise)
plt.plot(data)
# track_0 = np.array( [1259.3333333333333, 1258.2222222222222, 1271.6666666666667, 1241.7777777777778, 1258.0, 1225.2222222222222, 1214.7777777777778, 1247.7777777777778, 1230.6666666666667, 1231.4444444444443, 1205.111111111111, 1200.0, 1213.2222222222222, 1216.888888888889, 1210.3333333333333, 1231.2222222222222, 1234.888888888889, 1220.4444444444443, 1226.3333333333333, 1207.7777777777778, 1234.5555555555557, 1217.3333333333333, 1224.3333333333333, 1184.4444444444443, 1212.888888888889, 1177.8, 1178.0, 1164.3333333333333, 1231.7777777777778, 1212.3333333333333, 1203.1, 1214.5555555555557, 1208.1, 1195.111111111111, 1207.7777777777778, 1210.2222222222222, 1201.3333333333333, 1196.7777777777778, 1188.888888888889, 1214.7777777777778, 1212.2222222222222, 1230.3, 1202.2, 1210.5555555555557, 1202.5555555555557, 1174.7777777777778, 1208.6666666666667, 1216.5555555555557, 1227.888888888889, 1227.5555555555557, 1209.7, 1216.4444444444443, 1174.4, 1200.5555555555557, 1212.3333333333333, 1180.111111111111, 1218.3333333333333, 1199.2222222222222, 1196.111111111111, 1219.7777777777778, 1210.888888888889, 1189.5, 1222.4444444444443, 1206.5555555555557, 1201.888888888889, 1206.111111111111, 1162.7777777777778, 1206.3333333333333, 1183.3333333333333, 1225.6666666666667, 1212.4444444444443, 1185.5555555555557, 1210.5555555555557, 1188.888888888889, 1168.111111111111, 1171.111111111111, 1218.111111111111, 1230.5555555555557, 1209.5555555555557, 1207.6666666666667, 1189.3, 1215.0, 1206.3333333333333, 1194.2, 1217.111111111111, 1206.5555555555557, 1212.2222222222222, 1187.5555555555557, 1200.7777777777778, 1208.6666666666667, 1197.4444444444443, 1189.0, 1179.2222222222222, 1206.7777777777778, 1237.2222222222222, 1199.2222222222222, 1196.6666666666667, 1200.7777777777778, 1170.0, 1184.7777777777778, 1201.2222222222222, 1228.0, 1212.125, 1201.111111111111, 1192.8181818181818, 1199.0, 1198.4444444444443, 1198.888888888889, 1163.8181818181818, 1188.5555555555557, 1187.5555555555557, 1236.3333333333333, 1191.111111111111, 1210.6666666666667, 1194.9, 1237.888888888889, 1233.3333333333333, 1205.1, 1171.6666666666667, 1198.0, 1190.7777777777778, 1166.111111111111, 1188.111111111111, 1194.2222222222222, 1198.3333333333333, 1190.0, 1235.6666666666667, 1190.1, 1204.888888888889, 1195.7777777777778, 1200.625, 1193.7777777777778, 1199.6666666666667, 1203.6666666666667, 1201.0, 1206.4444444444443, 1211.2222222222222, 1195.2222222222222, 1195.3333333333333, 1247.5555555555557, 1220.0, 1180.2222222222222, 1224.111111111111, 1184.0, 1189.7777777777778, 1192.2222222222222, 1219.6666666666667, 1202.5555555555557, 1184.111111111111, 1167.2222222222222, 1195.111111111111, 1202.7777777777778, 1196.888888888889, 1185.4444444444443, 1191.7777777777778, 1184.6666666666667, 1200.5555555555557, 1203.0, 1193.4444444444443, 1205.111111111111, 1202.6666666666667, 1197.888888888889, 1197.3333333333333, 1181.0, 1241.0, 1215.0, 1175.5555555555557, 1204.888888888889, 1201.7777777777778, 1181.111111111111, 1175.5555555555557, 1209.6666666666667, 1205.4444444444443, 1197.3333333333333, 1196.3333333333333, 1171.4444444444443, 1181.3333333333333, 1204.3333333333333, 1166.0, 1187.7777777777778, 1200.888888888889, 1165.5555555555557, 1197.6666666666667, 1158.4444444444443, 1214.111111111111, 1197.5555555555557, 1159.5555555555557, 1220.6666666666667, 1176.6666666666667, 1194.7777777777778, 1210.0, 1199.6666666666667, 1224.0, 1194.7777777777778, 1202.4444444444443, 1180.2222222222222, 1207.7777777777778, 1186.888888888889, 1200.0, 1189.888888888889, 1173.7777777777778, 1235.3333333333333, 1205.0, 1200.2222222222222, 1175.6666666666667, 1194.2222222222222, 1206.4444444444443, 1222.7777777777778, 1201.5555555555557, 1166.6666666666667, 1222.5555555555557, 1199.3333333333333, 1180.4444444444443, 1198.7777777777778, 1186.6666666666667, 1177.5555555555557, 1200.7777777777778, 1190.0, 1194.4, 1186.4444444444443, 1191.111111111111, 1204.0, 1177.7, 1168.7777777777778, 1179.2222222222222, 1158.3, 1195.6666666666667, 1170.3333333333333, 1181.3333333333333, 1184.7777777777778, 1181.3333333333333, 1218.111111111111, 1204.7777777777778, 1191.2222222222222, 1185.2222222222222, 1186.2222222222222, 1190.4444444444443, 1201.0, 1182.111111111111, 1207.2222222222222, 1173.5555555555557, 1201.2222222222222, 1185.0, 1208.7777777777778, 1170.888888888889, 1188.7777777777778, 1219.4444444444443, 1176.111111111111, 1182.888888888889, 1191.888888888889, 1197.888888888889, 1162.111111111111, 1196.3333333333333, 1166.111111111111, 1149.4444444444443, 1190.4, 1175.6666666666667, 1170.7777777777778, 1182.888888888889, 1202.2222222222222, 1174.6, 1181.5555555555557, 1205.4444444444443, 1200.6666666666667, 1206.888888888889, 1187.4444444444443, 1188.888888888889, 1194.3333333333333, 1202.111111111111, 1202.4444444444443, 1222.0, 1190.2222222222222, 1178.2222222222222, 1184.111111111111, 1198.2222222222222, 1210.5555555555557, 1198.3333333333333, 1177.7777777777778, 1180.3333333333333, 1202.0, 1204.7777777777778, 1172.7777777777778, 1190.7777777777778, 1208.4444444444443, 1197.888888888889, 1184.3333333333333, 1209.2222222222222, 1179.888888888889, 1159.5, 1189.5555555555557, 1172.3333333333333, 1172.3333333333333, 1189.3333333333333, 1202.4444444444443, 1209.2222222222222, 1189.2222222222222, 1167.4444444444443, 1232.111111111111, 1203.5555555555557, 1183.6666666666667, 1172.3333333333333, 1209.6666666666667, 1185.2222222222222, 1180.111111111111, 1195.888888888889, 1197.0, 1161.888888888889, 1157.888888888889, 1200.7777777777778, 1175.5555555555557, 1189.2222222222222, 1186.2222222222222, 1185.888888888889, 1204.4444444444443, 1179.2222222222222, 1201.3333333333333, 1188.2222222222222, 1184.888888888889, 1204.0, 1183.6666666666667, 1173.2222222222222, 1173.3333333333333, 1149.3333333333333, 1199.888888888889, 1196.4444444444443, 1189.7777777777778, 1188.2222222222222, 1190.4444444444443, 1188.5555555555557, 1179.888888888889, 1218.4444444444443, 1187.3333333333333, 1203.0, 1192.3333333333333, 1178.888888888889, 1183.888888888889, 1169.7, 1192.7777777777778, 1175.2222222222222, 1177.0, 1182.3333333333333, 1183.2222222222222, 1190.5555555555557, 1170.6666666666667, 1195.4444444444443, 1149.7777777777778, 1188.111111111111, 1158.4444444444443, 1189.2222222222222, 1165.7777777777778, 1181.3333333333333, 1196.5555555555557, 1158.888888888889, 1172.0, 1201.2222222222222, 1172.3333333333333, 1182.888888888889, 1177.5555555555557, 1178.0, 1167.7777777777778, 1178.5555555555557, 1168.0, 1185.7777777777778, 1177.3333333333333, 1198.7777777777778, 1168.7777777777778, 1175.4444444444443, 1181.4444444444443, 1150.111111111111, 1173.888888888889, 1174.5555555555557, 1168.888888888889, 1151.111111111111, 1181.7777777777778, 1187.6666666666667, 1166.5555555555557, 1171.2222222222222, 1189.6666666666667, 1177.9, 1165.4444444444443, 1162.888888888889, 1167.5555555555557, 1175.2222222222222, 1163.6666666666667, 1165.0, 1162.7777777777778, 1203.5555555555557, 1196.7777777777778, 1142.5555555555557, 1159.2222222222222, 1177.8, 1186.25, 1181.2222222222222, 1169.4, 1161.4444444444443, 1173.4444444444443, 1184.111111111111, 1191.6666666666667, 1185.2222222222222, 1153.4, 1173.0, 1190.4444444444443, 1177.4444444444443, 1158.8, 1194.2222222222222, 1156.4444444444443, 1170.111111111111, 1191.0, 1177.7777777777778, 1175.111111111111, 1159.1, 1154.2222222222222, 1183.2222222222222, 1168.2222222222222, 1181.0, 1173.7777777777778, 1186.2222222222222, 1172.4444444444443, 1178.888888888889, 1166.5555555555557, 1174.5555555555557, 1167.2, 1178.888888888889, 1182.111111111111, 1187.111111111111, 1188.6666666666667, 1188.6666666666667, 1201.3333333333333, 1181.111111111111, 1168.888888888889, 1174.7777777777778, 1219.6666666666667, 1163.2222222222222, 1173.4444444444443, 1168.25, 1168.1, 1191.111111111111, 1168.0, 1145.6666666666667, 1190.6666666666667, 1185.0, 1207.25, 1173.6666666666667, 1183.3333333333333, 1186.111111111111, 1196.0, 1187.0, 1175.888888888889, 1160.4444444444443, 1149.888888888889, 1162.6666666666667, 1219.4444444444443, 1176.6666666666667, 1191.888888888889, 1187.5555555555557, 1190.2222222222222, 1156.5555555555557, 1185.7777777777778, 1189.0, 1196.375, 1183.5555555555557, 1187.0, 1187.0, 1195.5555555555557, 1172.4444444444443, 1176.7777777777778, 1184.888888888889, 1198.5555555555557, 1193.3333333333333, 1209.0, 1198.3333333333333, 1181.2222222222222, 1191.5555555555557, 1191.7777777777778, 1167.6666666666667, 1171.1, 1172.888888888889, 1184.888888888889, 1183.888888888889, 1172.5555555555557, 1184.888888888889, 1186.5555555555557, 1174.111111111111, 1181.3333333333333, 1179.2, 1175.3333333333333, 1185.3333333333333, 1204.3333333333333, 1191.7777777777778, 1181.3333333333333, 1194.5555555555557, 1201.0, 1173.7333333333333])


In [224]:
# https://static-content.springer.com/esm/art%3A10.1038%2Fnature04928/MediaObjects/41586_2006_BFnature04928_MOESM3_ESM.pdf
#The location (loc) keyword specifies the mean. The scale (scale) keyword specifies the standard deviation.
mean_data = np.mean(data_noise)
std_data = np.std(data_noise)
guassian_data = norm.pdf(data_noise, loc = mean_data, scale= std_data)
print(guassan_data)
print(data_noise)
print(std_data)
print(len(data_noise))

[0.00228833 0.00318837 0.00352328 ... 0.00377356 0.00311736 0.00291683]
[-17.58322995   8.86497065  18.90504138 ... 140.94460973 160.98170909
 166.82801352]
48.72131902707007
20000


In [226]:
from scipy.optimize import curve_fit
from scipy.stats import chi2

def gaussian_fit(X):
    
    def _gauss(x, A, x0, sigma):
        return A * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

    x_data = np.arange(len(X))

    mean_data = np.sum(x_data * X) / np.sum(X)
#     sigma = np.sqrt( np.sum(data_noise * (x_data-mean_data)**2 ) / np.sum(X) ) #std
    sigma = 8.0/0.5

    parameters, covariance = curve_fit(_gauss, x_data, X, p0= [np.max(X), mean_data, sigma])
    print(parameters)
    data_x = np.array(list(range(len(data_noise))))
    # print(type(data_x))
    fitted_data = _gauss(data_x, *parameters)
    return fitted_data

data_noise_normalize = NormalizeData(data_noise)
data_norm = NormalizeData(data)
# fitted_data_norm = gaussian_fit(data_noise_normalize)
# fitted_data = gaussian_fit(data_noise)
# print(fitted_data)
# print(data_noise)

plt.plot(data_noise)
# plt.plot(fitted_data)
plt.plot(data)

In [212]:
def _chi2(x, E):
    return np.sum((x-E)**2/E)
x_noise_data = np.arange(len(data_noise))
mean_data_noise = np.sum(x_noise_data * data_noise) / np.sum(data_noise)
sigma_noise = np.sqrt( np.sum(data_noise * (x_noise_data-mean_data_noise)**2 ) / np.sum(data_noise) ) #std
parameters, covariance = curve_fit(_chi2, x_noise_data, data_noise, p0=fitted_data)
print(parameters)

TypeError: _chi2() takes 2 positional arguments but 20001 were given

In [225]:
from scipy.stats import chisquare
from scipy.stats import sem

def chi2_custom(O,E):
    result = []
    for index in range(len(O)):
        result.append((O-E[index])**2/E[index])
    return result
chi2_r = chi2_custom(data_noise, guassian_data)
print(np.where(chi2_r == np.min(chi2_r)))

def residuals_chi(O, E):
    return (O-E)/ np.sqrt(E)

def chisquare_custom(O,E):
#     std_err = np.array([np.std(i)/np.sqrt(2) for i in zip(O,E)])
    std_err = sem(O)
    print(np.sum( ((O-E)/std_err)**2 ))

def std_residuals_chi(O, E):
    std_o = np.std(O)
    chival =(O-E)/std_o
    return chival

# r_chi = residuals_chi(data_noise, fitted_data)
# std_r_chi = std_residuals_chi(data_noise, fitted_data)
# # chisquare_custom(data_noise, fitted_data)
# # from scipy.stats import chi2
# # print(len(data_noise))
# # print(chi2.isf(16, len(data_noise)-1))
# # r_chi_norm = residuals_chi(data_noise_normalize, fitted_data)
# # print(np.max(r_chi))
# print(np.where(r_chi == np.min(r_chi)))
# print(np.where(std_r_chi == np.min(std_r_chi)))
# print(np.where(r_chi == np.max(r_chi)))
# print(np.where(r_chi_norm == np.min(r_chi_norm)))

(array([10468], dtype=int64), array([1979], dtype=int64))


In [ ]:
# https://www.scribbr.com/statistics/chi-square-goodness-of-fit/
# https://heytutor.com/resources/blog/how-to-calculate-p-value-using-chi-square/

Signoficance_level = 0.05 #SL
chi_data = np.array([[22,25], [30,25], [23,25]])

# chi2.fit(chi_data)
f_obs = chi_data[:,0]
f_exp = chi_data[:,1]
chisquare(f_obs=[22], f_exp=[25])

# if pval <= SL hypothesis is accepted
# else pyhothesis is rejected

In [ ]:
# https://scholarworks.umass.edu/cgi/viewcontent.cgi?article=1269&context=pare
# A residual is the
# difference between the observed and expected values
# for a cell. The larger the residual, the greater the
# contribution of the cell to the magnitude of the
# resulting chi-square obtained value
# Std Residual  = O -  E / sqrt(E)
# O = observed E = Expected

In [107]:
xdata = [ -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
ydata = [1.2, 4.2, 6.7, 8.3, 10.6, 11.7, 13.5, 14.5, 15.7, 16.1, 16.6, 16.0, 15.4, 14.4, 14.2, 12.7, 10.3, 8.6, 6.1, 3.9, 2.1]
  
# Recast xdata and ydata into numpy arrays so we can use their handy features
xdata = np.asarray(xdata)
ydata = np.asarray(ydata)
plt.plot(xdata, ydata, 'o')

In [ ]:

resudual_chi = 